In [ ]:
import datetime

import fiona
from shapely.geometry import shape, mapping
import xarray as xr
import rioxarray

from nrt.validate.interface import SegmentsLabellingInterface
from nrt.validate.loaders import FileLoader
from nrt.validate.webmaps import EsriWaybackBasemap, PlanetBasemap, GoogleBasemap
from nrt.validate import utils
from nrt.validate.indices import *
from nrt.validate.composites import *
from nrt.validate.xr_transforms import *
from nrt import data

# Feature collection from nrt.data package (300 features; Point geometry; unique property: fid)
fc = data.germany_sample_points()
fc = sorted(fc, key=lambda feature: feature['properties']['fid'])


# Webamp for VHR panel; can also be planet, google, etc
webmap = EsriWaybackBasemap().map

# Vegetation indices for time-series plot
vis = {'NDVI': utils.combine_transforms(S2CloudMasking(), NDVI(nir='B08')),
       'CR-SWIR': utils.combine_transforms(S2CloudMasking(), CR_SWIR(nir='B08')),
       'NCDI': utils.combine_transforms(S2CloudMasking(), NCDI(nir='B08'))}

# Chip size in CRS unit
window_size = 500

# Color composite
compositor = S2CIR(r='B08', blim=[0.025, 0.13], glim=[0.0150, 0.1700], rlim=[0.1500, 0.4000])

# Sample unit resolution (mandatory if Point geometry)
res = 10.0

# Input data source (online zarr store treatead as filesystem file in that case
cube = data.germany_zarr()
cube = cube.sel(time=slice(datetime.datetime(2019,1,1), datetime.datetime(2021,12,31)))

# Additional optional pre-processor passed to xr_transform
def drop_nan_slices(ds):
    """
    Drops all time slices of an xarray.Dataset where the B02 variable 
    is composed entirely of NaN values.
    """
    nan_slices = ds['B02'].isnull().all(dim=['x', 'y'])
    filtered_ds = ds.sel(time=~nan_slices)
    return filtered_ds

# Loader to prepare data in a standardized way for interface (See STACLoader for alternative input data source)
loader = FileLoader(fc=fc,
                    key='fid',
                    crs=cube.rio.crs,
                    datasets=cube,
                    vis=vis,
                    window_size=window_size,
                    compositor=compositor,
                    xr_transform=drop_nan_slices,
                    res=res)


# Interface
i = SegmentsLabellingInterface(loader=loader,
                               webmap=webmap,
                               res=res,
                               labels=['Non-treed',
                                       'Stable tree cover',
                                       'Dieback',
                                       'Regrowth'])

In [ ]:
i.display()